This program processes and plots example output from GP Microbiome using functions and loops, then saves the results. Similar to Plotsamples, which plots ouput from GP Microbiome with all time points included, this program is written for GP Microbiome output which results from inputting data without the values from at least one time point, then making a prediction for that time point(s) to test the accuracy of the predictions. Originally I used CF study participant 708's data and withheld the last time point, then predicted the last time point and 4 time points into the future. For this example, I followed a similar pattern with randomly generated data, created from two of my Plotsamples example files. I show how I created these files in Section 4.
<br>

For the first file, I once again predicted the last time point, but this time predicted only 3 in the future. For the second, I included predictions between time points and excluded both the second and last time points, then also predicted 3 in the future. See Plotsamples for a complete explanation of those example files, which take the form of GP Microbiome output and observed relative abundance data, and how I have provided them in the repository.  
<br>

Like those in Plotsamples, the functions in this program produce as many as twenty plots for each participant, and when they are run in a loop they generate plots for all participants at once. The output data is plotted with the observed relative abundance data in a similar way, and the plots use the same colour-coded markers to indicate the participant's clinical condition at each time point. Additionally, yellow markers indicate the prediction(s) for the withheld time point(s), to make it easy to see the level of accuracy. 
<br>

I have also included comments indicating what the original code looked like, as the only differences are file names and the fact that the original data were not randomly generated.
<br>

This code can also easily be adapted to plot output from other algorithms in situations where predictions have been made and validated with a leave-one-out (or leave-k-out, for any value of *k* ) scenario.

In [ ]:
#import necessary libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Section 1
The first few cells create the OTUkey_named file. If the file has already been created, you can skip down to Section 2.

In [ ]:
#Read in the original key file. We are going to add a column for only the bacteria's genus name to it.
key = pd.read_excel("Data/OTUkey.xlsx")
#rename first column to avoid Excel mistaking it for a SYLK file due to the "ID" in the name
key.rename(columns={'ID_OTU': 'OTU'}, inplace=True)
#view the head, to get an idea of the rest of the format
key.head()

In [ ]:
#extract the genus from the taxonomic information and create the new 'Name' column for it
pat = 'D_5__(?P<Name>.*)'
key=key.join(key.Bacteria.str.extract(pat, expand=True))

In [ ]:
#replace NaN, which occurs where the genus is "Other", with the word "Other"
key['Name'].fillna('Other', inplace=True)
#save edited file and review changes
key.to_csv("Data/OTUkey_named.csv", index=False)
key.head()

## Section 2
Read in the OTUkey_named file below if you have previously created and saved it with Section 1. I have included a copy of it in the 'Extras' folder as well.

In [ ]:
#read in OTUkey_named file, if it has already been created 
key=pd.read_csv("Data/OTUkey_named.csv")

In [ ]:
#make a list of the operational taxonomic unit (OTU) IDs for our bacteria of interest
bacteria=[2,30,58,59,60,63,70,80,94,104,113,167,169,170,206,221,223,227,229,234]

## Section 3
This section creates a second version of the OTUkey_named file for selected bacteria for use in other programs, and can also be skipped once you have the file. We will use a version of the OTUkey_named_selection file, with plot specifcations added, in DTW_All_boxplots, where we create boxplots of the TIME Dynamic Time Warping output.

In [ ]:
#this cell can be skipped if the OTUkey_selection_named file already exists, as it is not used in this particular program.
#creating a second key for selected bacteria
selectkey=key.iloc[[i-1 for i in bacteria],:]
selectkey.head()
#saving the select key for later use in other programs
selectkey.to_csv("Data/OTUkey_selection_named.csv", index=False)

## Section 4: How I generated the additional example output files

To generate new data, first we read in our example output file for fictional participants 480 and 453. These example files are intended to resemble real output and are not actually the results of running GP Microbiome. Had they been, they would have been generated from the raw output in the program readsample27. I have included in this repository both the example files and a full explanation of how I created them, for those who are interested. The code here to generate new versions of those files is very similar to the code I used when I originally generated them, but I took additional steps in that initial process to make a more realistic approximation of our data than I would have had if I'd used a completely random process. 
<br> 

This means that we can easily create new versions of the filest that are still realistic simply by applying random weights to the values and normalizing them to sum to 1. For the new version of 480's data set, I only predict the last time point, along with 3 future ones. For the new version of 453's data set, I include predictions between time points and withhold both the second and last time point, predicting for them as well. 

I demonstrate two different techniques below for generating new data: for 480, I read in the file which does not contain predictions, apply weights and normalize the existing data and generate my predictions (for the withheld final time point and future time points) based on the withheld final time point. Then I combine the tables to create the file which includes 'predictions.' For 453, I reverse the process and read in the file with predictions included, apply weights and normalize, then cut the 'predictions' out to create a new version of 453's prediction-free file. 

First we generate the file for participant 480:

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
df=pd.read_csv("Data/480.csv")
#view the file, to get an idea of what it looks like
df.head()

In [ ]:
#convert to an array, apply the random weights and normalize, then preview the new array
np.random.seed(1)
c = normalize(df.iloc[1:,:10].to_numpy()*np.random.uniform(low=.5, high=1.5, size=(245,len(df.columns)-1)), axis=0, norm='l1')
c

In [ ]:
#convert back to a data frame, adding the first row back in (without the last column)
df2=pd.DataFrame(c)
df3=pd.DataFrame(np.insert(df2.values, 0, values=df.iloc[0,:10].to_list(), axis=0))
df3.to_csv('Data/480b.csv', index=False)
#view result
df3.head()

In [ ]:
#add predictions - I chose to make 3 in the future this time, so we have a total of 4
np.random.seed(2)
y=df.iloc[1:,10].to_numpy()
x=normalize(y.repeat(4).reshape((245,4))*np.random.uniform(low=.5, high=1.5, size=(245,4)),axis=0, norm='l1')
#alternative code, with same result using np.resize instead of np.reshape
#x=normalize(np.resize(y.T,(4,245)).T*np.random.uniform(low=.5, high=1.5, size=(245,4)),axis=0, norm='l1')

In [ ]:
#Combine the two data frames. Check the head to confirm that it looks right.
#I don't reorder the columns yet.
#This makes it possible to plot or otherwise analyse predicted values on their own without creating a separate file
#Note that for this example I didn't predict between time points, so this will be in order anyway
pred_df=pd.DataFrame(x)
pred_df2=pd.DataFrame(np.insert(pred_df.values, 0, values=[180*i+df.iloc[0,10] for i in range(4)], axis=0),columns=[i for i in range(len(df.columns)-1, len(df.columns)+3)])
dfboth = pd.concat([df3, pred_df2], axis=1, sort=False)
dfboth.head()

In [ ]:
dfboth.to_csv("Data/480b_both.csv", index=False)

Now we generate the file for participant 453:

In [ ]:
df=pd.read_csv("Data/453_both.csv")
#view the head of the file, noting the original time points
df.head()

In [ ]:
#convert to an array, apply the random weights and normalize, then preview the new array
np.random.seed(3)
c = normalize(df.iloc[1:,:].to_numpy()*np.random.uniform(low=.5, high=1.5, size=(245,len(df.columns))),axis=0, norm='l1')
c

In [ ]:
df2=pd.DataFrame(c)
df3=pd.DataFrame(np.insert(df2.values, 0, values=df.iloc[0].to_list(), axis=0))
df3.to_csv('Data/453b_both.csv', index=False)
#since 453_both.csv (like all similarly named files) has the actual time points on the left and the predictions on the right,
#separating out the predictions to create a new file without predictions in it is easy
#remove the second column, the last column in 453_both.csv that was not a prediction, and the predictions columns
#then rename the columns, numbering them with their index values, and save
df3.iloc[:,:7].drop(df3.iloc[:,:7].columns[1], axis=1).set_axis([i for i in range(6)], axis=1, inplace=False).to_csv('Data/453b.csv', index=False)

## Section 5: Plots
Read the files into dictionaries and plot them with functions. The dictionaries are to facilitate plotting multiple particpants' output at once in loops. If you only have one file, you can run the function on just that participant ID or create a 1-item dictionary. 
<br>

See Section 6 for the code to create legends, with options to save the legends as separate files (my preferred method for the CF output data) or to copy and paste into any of the functions in Section 5 at the indicated places.

## Part A: Read in files

In [ ]:
#create a list for the ID numbers of the participants whose data we ran through GP Microbiome with time point(s) excluded
IDs=['480','453']
#Create dictionaries and read in each person's leave-k-out output for noise-free compositions without predictions, 
#and with predictions added in. We showed how we generated new example files for 480 and 453 in Section 4. 
#recall that my files for leave-k-out data differ in name from the ones for data with all time points only by the letter 'b' 
#I kept up this convention of simply adding a 'b' in the output from the program as well.
dfs = {i: pd.read_csv('Data/{}b.csv'.format(i)) for i in IDs}
both_dfs = {i: pd.read_csv('Data/{}b_both.csv'.format(i)) for i in IDs}
#we need the files with all time points included too, to create a dictionary indicating which time point(s) were omitted
observed_dfs = {i: pd.read_csv('Data/{}.csv'.format(i)) for i in IDs}

In [ ]:
#if we were running this on the CF Data, we would simply change the IDs list:
#IDs=['708']

In [ ]:
#if desired, view the first few entries for one of the files without predictions to get a feel for the data
dfs['480'].head()  

In [ ]:
#rename the columns in the files containing both sets of time points based on the first row, which contains the time points
#reorder the columns in the files to make the time points consecutive, and put them in a new dictionary
#for 480 they are already in order because we did not predict between time points, but 453 will need to be reordered
reordered_dfs={}
for i in IDs:
    df=both_dfs[i].set_axis(both_dfs[i].loc[0].tolist(), axis=1, inplace=False)
    df=df.reindex(columns=sorted(df.columns))
    #save file if desired
    #df.to_csv('Data/{}b_both_reordered.csv'.format(i), index=False)
    #if you did save it, you could edit the first cell in Part A to read it directly into a dictionary 
    #I opted not to do so because it takes so little time to reorder and I wanted to save space on my computer
    #I wanted the unordered version saved so that I could easily examine predicted values on their own 
    reordered_dfs[i]=df

In [ ]:
#view the resulting reordered data frame, confirming that the redordering code ran correctly 
reordered_dfs['453'].head()

The next cell reads in files created in the program Create_relative_abundance_files. If you have not run that program yet, edit the cell as directed in the comments to use the copy in the 'Extras' folder.

In [ ]:
#read in the files containing the observed relative abundance data for each participant, adding them to a new dictionary
#the columns are the age in days at the time of each sample, and we will use this information as well in the plots
#the files were created and saved in the program Create_relative_abundance_files
#however, they are also in the 'Extras' folder for your convenience
rel_dfs = {i: pd.read_csv("Data/{}_Rel.csv".format(i)) for i in IDs}
#to use the ones in the 'Extras' folder, comment out the previous line and un-comment this one:
#rel_dfs = {i: pd.read_csv("Data/Extras/Relative Abundance Files/{}_Rel.csv".format(i)) for i in IDs}

In [ ]:
#if desired, examine the head of one of the files to get a feel for the data
rel_dfs['480'].head()

## Part B: Markers
In our plots, different coloured markers indicate a participant's clinical condition at each of the time points where samples were taken. In plots without predictions, every time point is marked. In plots with predictions, predicted time points are of course not marked. This means that markers need to be customised, and that having varying numbers of predictions between time points can make the markers especially tricky to define.
<br>

I did predict between time points as well as into the future when I ran GP Microbiome with data for all time points included, inserting 1 or 2 at even intervals. For the leave-k-out plots, actual participant 708 and example participant 480 only had predictions for the last time points and future ones but example participant 453 followed the same pattern of having varying numbers of predictions between time points. 
<br>

Because we can exclude *k* time points for any value of _k_, and we can do so in any positions, the most straightforward way to create markers for these plots is to use two sets of markers. This avoids confusion and makes it easy to put yellow markers on the observed time point(s) which we predicted for. It also allows for more readable code. 
<br>

There are two main ways of creating dictionaries for the markers: One is to import metadata and process it into dictionaries directly. The other is, after doing the first method once and saving the results to an Excel file (in my case, the same metadata file), to import those results into dictionaries. Since we are using an edited version of the data set, and since one might want to do these kinds of plots multiple times with different time points excluded, it doesn't make sense to save these markers and read them in. It's more efficient to create them directly, adding new dictionaries for the leave-k-out data and the withheld time points based on the metadata. 

In [ ]:
#read in the metadata file which includes the condition and time delta for each participant's samples 
status=pd.read_excel("Data/ExampleDeltaKey.xlsx", sheet_name="Metadata and time deltas")
#for the CF Data, the name of the file is 'MetaDataKey.xlsx'
#otherwise, the code is identical

In [ ]:
#create a dictionary which, for each participant, lists the time deltas for samples taken while stable
S_list={}
for i in [int(x) for x in IDs]:
    #convert to a list, for each ID, the entries in the Time_Delta column for which the Visit_type was 'Stable'
    S_list[i]=list(status.query('Participant == {} and Visit_type == "Stable"'.format(i))['Time_Delta'])
#display to confirm
S_list

In [ ]:
#create a dictionary which, for each participant, lists the time deltas for samples taken during exacerbations
E_list={}
for i in [int(x) for x in IDs]:
    #convert to a list, for each ID, the entries in the Time_Delta column for which the Visit_type was 'Exacerbation'
    E_list[i]=list(status.query('Participant == {} and Visit_type == "Exacerbation"'.format(i))['Time_Delta'])
#display to confirm
E_list

In [ ]:
#create a dictionary of lists of the withheld time points for each participant
out_dict={}
for name in IDs:
    out_dict[name]=[i for i in list(observed_dfs[name].iloc[0]) if i not in list(dfs[name].iloc[0])]
out_dict

In [ ]:
#make a dictionary depicting the order of the values for each status, to identify where to place markers
#then make a version of the dictionary for the leave-k-out file
#these are for use with plots without predictions
#these positional values correspond to column names in the 'observed_dfs' dictionary, which are simple index values 
#if they weren't numbers, we would replace int(col) with dfs[name].columns.get_loc(col)
#a 'g' or 'r' in the dictionary name indicates green or red, for stable or exacerbated status, respectively
#the 'new' prefix indicates that a dictionary is for leave-k-out data
markers_gdict={}
new_gdict={}
markers_rdict={}
new_rdict={}
for i in IDs:
#make a dictionary depicting the order of the values in the lists contained in S_list, which will be for green markers
    markers_gdict[i]=[int(col) for col in observed_dfs[i].columns if observed_dfs[i][col][0] in S_list[int(i)]]
#we also have a version of the dictionary for the leave-k-out file
    new_gdict[i]=[int(col) for col in dfs[i].columns if dfs[i][col][0] in S_list[int(i)] and dfs[i][col][0]
                 not in out_dict[i]]
#make a dictionary depicting the order of the values in the lists contained in E_list, which will be for red markers
    markers_rdict[i]=[int(col) for col in observed_dfs[i].columns if observed_dfs[i][col][0] in E_list[int(i)]]
#we also have a version of the dictionary for the leave-k-out file
    new_rdict[i]=[int(col) for col in dfs[i].columns if dfs[i][col][0] in E_list[int(i)] and dfs[i][col][0]
                 not in out_dict[i]]
#display to confirm that this is consistent with the values we removed
markers_gdict, markers_rdict, new_gdict, new_rdict

In [ ]:
#make versions of the dictionaries for the plots with predictions for the leave-k-out files
#these correspond to index values of column names in the 'reordered_dfs' dictionary
#which are time deltas for both actual and predicted values
new_g1dict={}
new_r1dict={}
markers_ydict={}
for i in IDs:
#first make a dictionary depicting the order of the values in the lists contained in S_list, which will be for green markers
    new_g1dict[i]=[reordered_dfs[i].columns.get_loc(col) for col in reordered_dfs[i].columns 
                    if reordered_dfs[i][col][0] in S_list[int(i)] and reordered_dfs[i][col][0] not in out_dict[i]]
#next make a dictionary depicting the order of the values in the lists contained in E_list, which will be for red markers
    new_r1dict[i]=[reordered_dfs[i].columns.get_loc(col) for col in reordered_dfs[i].columns 
                    if reordered_dfs[i][col][0] in E_list[int(i)] and reordered_dfs[i][col][0] not in out_dict[i]]
#finally, make a dictionary to mark the predictions for the withheld points, which will have square yellow markers
    markers_ydict[i]=[reordered_dfs[i].columns.get_loc(col) for col in reordered_dfs[i].columns 
                      if reordered_dfs[i][col][0] in out_dict[i]]    
#display to confirm that all the markers are in the right places
new_g1dict, new_r1dict, markers_ydict

If you were to decide to save your markers to a file, you could insert here the same code used in Plotsamples and edit it easily to do so. I would recommend making a new sheet just for the leave-k-out markers, to avoid confusion.

## Part C: Creating the plots

My plotting functions create as many as 20 plots per participant, and when run in loops they plot all participants' data at the same time. Before running such a function, always make sure that your input data is formatted consistently for each participant, to ensure that the plots show what they are intended to show.
<br>

All of my plotting functions are written to save the plots to a folder called 'Plots,' which is in this repository as well. Adjust the file path if you want to save them somewhere else, or comment out the line of code which saves them. 

In [ ]:
#define custom colours for the plots - light and dark red and green, for noise-free and observed values respectively
l_red='#FF5959'
d_red='#A40000'
l_green='#14AE0E'
d_green='#0B5A08'

In [ ]:
#Given the distribution of our data, it didn't make sense to define the y-axis the same way for all the plots. 
#However, if you did wish to do so, you would add the following code to the plot specifications section of the function:
#plt.ylim(min_value, max_value) 
#and substitute in the minimum and maximum values you wish to use

First, we have a function for use in a loop with the dictionaries, plotting without predictions. I included the observed values for the omitted time points in these plots (on the solid line) for reference. Although you may not feel the need to save these, you may want to see them to note how they differ from the corresponding plots using all time points. Because of how GP Microbiome works, output from a smaller input data set will look similar but not identical at each common time point, and seeing how things change provides insight about the withheld data.
<br>

Since our leave-k-out example output data were randomly generated, these plots may not look as similar to the plots of all of the original example output data as they would have if they were actually the results of running GP Microbiome.
<br>

These plots only differ from the first parts of the plots with predictions if you predicted between time points. Plots with predictions follow.

In [ ]:
#function for use in a loop with the dictionaries, plotting noise-free compositions without predictions
def plot_loop(name):
    #divide the list of bacteria of interest into groups of 4 to facilitate plotting
    rows=[[2,30,58,59],[60,63,70,80],[94,104,113,167],[169,170,206,221],[223,227,229,234]]
    s=dfs[name]
    rel=rel_dfs[name]
    days=[int(x) for x in rel.columns]
    out_list=out_dict[name]
    markers_r = markers_rdict[name]
    markers_g = markers_gdict[name]
    new_r=new_rdict[name]
    new_g=new_gdict[name]
    ID=int(name) 
    #run a loop to plot each group of 4 in a 2 by 2 format with our custom markers, then save the file
    for j in range(5):
        fig = plt.figure(figsize=(18,14))
        for i in range(4):
            ax = fig.add_subplot(2,2,i+1)
            #because I made my markers slightly transparent, I need separate plots for lines and red markers
            #this avoids having the line become transparent
            #slightly transparent markers make it easier to see subtle differences between the lines 
            #if you opt to set alpha at the default of 1 (non-transparent), you can combine the first 2 red plots this way:
            #ax.plot([age for age in days if age-days[0] not in out_list], s.iloc[rows[j][i]],'-gD', 
                    #markevery=new_r, markerfacecolor=l_red, markersize=8, 
                    #linewidth=2,dashes=[2, 2,5,2], c='black')
            #there's no built-in way to customise marker colours by variables, so green markers need a dummy line 
            #we need to exclude the withheld time points in the plot for noise-free compositions of bacteria
            ax.plot([age for age in days if age-days[0] not in out_list], s.iloc[rows[j][i]],'-gD', 
                    markevery=new_r, markerfacecolor='none',markersize=8, linewidth=2,dashes=[2, 2,5,2], c='black')
            ax.plot([age for age in days if age-days[0] not in out_list], s.iloc[rows[j][i]],'-gD', 
                    markevery=new_r, markerfacecolor=l_red, alpha=0.75, markersize=8,c='none')
            ax.plot([age for age in days if age-days[0] not in out_list], s.iloc[rows[j][i]],'-gD', 
                    markevery=new_g, markerfacecolor=l_green,alpha=0.75, markersize=8, c='none')
            #for the observed values, we leave the withheld time points in for reference
            #again, if you prefer alpha=1 you can combine the two lines for red markers:
            #ax.plot(days,rel.iloc[rows[j][i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red, markersize=8, 
                    #linewidth=2, c='black')
            ax.plot(days,rel.iloc[rows[j][i]-1],'-gD', markevery=markers_r,markerfacecolor='none',markersize=8, 
                    linewidth=2, c='black')
            ax.plot(days,rel.iloc[rows[j][i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red,alpha=0.75, markersize=8, 
                    c='none')
            ax.plot(days,rel.iloc[rows[j][i]-1],'-gD', markevery=markers_g,markerfacecolor=d_green,alpha=0.75, markersize=8, 
                    c='none')
            #optional: insert code from Section 6 to add a legend for each plot - remember to make size/fit adjustments
            plt.title('{} Composition'.format(key['Name'][rows[j][i]-1]), size=15)
            plt.xlabel("Age (Days) of Participant {}".format(ID), size=13)
            plt.ylabel("Relative Abundance", size=13)
            plt.savefig("Plots/{}b_{}.png".format(ID,j), format='png')
        plt.show()

In [ ]:
#run the first function in a loop
for name in IDs:
    plot_loop(name)

All subsequent plots in this section contain predictions.

In [ ]:
#function to plot with predictions, for use in a loop with the dictionaries
def plot_pred_loop(name):
    #divide the list of bacteria of interest into groups of 4 to facilitate plotting
    rows=[[2,30,58,59],[60,63,70,80],[94,104,113,167],[169,170,206,221],[223,227,229,234]]
    r=reordered_dfs[name]
    rel=rel_dfs[name]
    days=[int(x) for x in rel.columns]
    out_list=out_dict[name]
    markers_r = markers_rdict[name]
    markers_g = markers_gdict[name]
    new_r1 = new_r1dict[name]
    new_g1 = new_g1dict[name]
    markers_y=markers_ydict[name]
    ID=int(name) 
    #run a loop to plot each group of 4 in a 2 by 2 format with our custom markers, then save the file
    for j in range(5):
        fig = plt.figure(figsize=(18,14))
        for i in range(4):
            ax = fig.add_subplot(2,2,i+1)
            #because I made my markers slightly transparent, I need separate plots for lines and red markers
            #this avoids having the line become transparent
            #slightly transparent markers make it easier to see subtle differences between the lines
            #if you opt to set alpha at the default of 1 (not transparent), you can combine the first two red plots:
            #ax.plot(r.loc[0]+days[0], r.iloc[rows[j][i]],'-gD', markevery=new_r1, 
                    #markerfacecolor=l_red,markersize=8, linewidth=2,dashes=[2, 2,5,2], c='black')  
            #there's no built-in way to customise marker colours by variables, so green and yellow markers always need a dummy line
            ax.plot(r.loc[0]+days[0], r.iloc[rows[j][i]],'-gD', markevery=new_r1, 
                    markerfacecolor='none', markersize=8, linewidth=2,dashes=[2, 2,5,2], c='black')
            ax.plot(r.loc[0]+days[0], r.iloc[rows[j][i]],'-gD', markevery=new_r1, 
                    markerfacecolor=l_red, alpha=0.75, markersize=8, c='none')
            ax.plot(r.loc[0]+days[0], r.iloc[rows[j][i]],'-gD', markevery=new_g1, 
                    markerfacecolor=l_green,alpha=0.75, markersize=8, c='none')
            #square yellow markers show the predictions for time points which were withheld
            ax.plot(r.loc[0]+days[0], r.iloc[rows[j][i]],'-gs', markevery=markers_y, 
                    markerfacecolor='yellow', alpha=0.9, markersize=11, c='none')
            #again, if you prefer alpha=1 you can combine the two lines for red markers:
            #ax.plot(days,rel.iloc[rows[j][i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red, markersize=8, 
                    #linewidth=2, c='black')
            ax.plot(days, rel.iloc[rows[j][i]-1],'-gD', markevery=markers_r,markerfacecolor='none',markersize=8, 
                    linewidth=2, c='black')
            ax.plot(days, rel.iloc[rows[j][i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red, alpha=0.75, markersize=8, 
                    c='none')
            ax.plot(days,rel.iloc[rows[j][i]-1],'-gD', markevery=markers_g,markerfacecolor=d_green,alpha=0.75, markersize=8, 
                    c='none')
            #optional: insert code from Section 6 to add a legend for each plot - remember to make size/fit adjustments
            plt.title('{} Composition with Predictions'.format(key['Name'][rows[j][i]-1]), size=15)
            plt.xlabel("Age (Days) of Participant {}".format(ID), size=13)
            plt.ylabel("Relative Abundance", size=13)
            plt.savefig("Plots/{}b_pred_{}.png".format(ID,j), format='png')
        plt.show()

In [ ]:
#run the function with predictions in a loop
for name in IDs:
    plot_pred_loop(name)

In [ ]:
#code to plot with predictions for the just 3 most important bacteria in a row
def plot_pred_rows(name):
    rows=[94,113,229]
    r=reordered_dfs[name]
    rel=rel_dfs[name]
    days=[int(x) for x in rel.columns]
    out_list=out_dict[name]
    markers_r = markers_rdict[name]
    markers_g = markers_gdict[name]
    new_r1 = new_r1dict[name]
    new_g1 = new_g1dict[name]
    markers_y=markers_ydict[name]
    ID=int(name) 
    fig=plt.figure(figsize=(26,7))
    for i in range(3):
        ax = fig.add_subplot(1,3,i+1)
        #because I made my markers slightly transparent, I need separate plots for lines and red markers
        #this avoids having the line become transparent
        #slightly transparent markers make it easier to see subtle differences between the lines
        #if you opt to set alpha at the default of 1 (not transparent), you can combine the first two red plots:
        #ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_r1, markerfacecolor=l_red,markersize=8, 
                    #linewidth=2,dashes=[2, 2,5,2], c='black')                
        #there's no built-in way to customise marker colours by variables, so the green markers always need a dummy line          
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_r1, markerfacecolor='none',markersize=8, 
                  linewidth=2,dashes=[2, 2,5,2], c='black')
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_r1, markerfacecolor=l_red, alpha=0.75, 
                markersize=8, c='none')
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_g1, markerfacecolor=l_green, alpha=0.75,
                markersize=8, c='none') 
        #square yellow markers show the predictions for time points which were withheld
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gs', markevery=markers_y, 
                markerfacecolor='yellow', alpha=0.9, markersize=11, c='none')
        #again, if you prefer alpha=1 you can combine the two lines for red markers:
        #ax.plot(days,rel.iloc[rows[i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red, markersize=8, 
                #linewidth=2, c='black')
        ax.plot(days, rel.iloc[rows[i]-1],'-gD', markevery=markers_r,markerfacecolor='none',markersize=8, 
                linewidth=2, c='black')
        ax.plot(days, rel.iloc[rows[i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red, alpha=0.75, markersize=8, 
                c='none')
        ax.plot(days,rel.iloc[rows[i]-1],'-gD', markevery=markers_g,markerfacecolor=d_green,alpha=0.75, markersize=8, 
                c='none')
        #optional: insert code from Section 6 to add a legend for each plot - remember to make size/fit adjustments
        plt.title('{} Composition with Predictions'.format(key['Name'][rows[i]-1]), size=24)
        plt.xlabel("Age (Days) of Participant {}".format(ID), size=18)
        plt.ylabel("Relative Abundance", size=18)
        plt.setp(ax.get_xticklabels(), size=14)
        plt.setp(ax.get_yticklabels(), size=14)
        #the tight_layout function reduces white space in the image. 
        #If you turn off tight_layout you may need to adjust your text size etc. 
        plt.tight_layout()        
        plt.savefig("Plots/{}b_pred_rows.png".format(ID), format='png')
    plt.show()

In [ ]:
#test-run the function on one participant
plot_pred_rows('453')

In [ ]:
#plotting with predictions for 2 in a row - simple edit to plot_pred_rows
def plot_pred_two(name):
    rows=[94,229]
    r=reordered_dfs[name]
    rel=rel_dfs[name]
    days=[int(x) for x in rel.columns]
    out_list=out_dict[name]
    markers_r = markers_rdict[name]
    markers_g = markers_gdict[name]
    new_r1 = new_r1dict[name]
    new_g1 = new_g1dict[name]
    markers_y=markers_ydict[name]
    ID=int(name) 
    fig=plt.figure(figsize=(15,6))
    for i in range(2):
        ax = fig.add_subplot(1,2,i+1)
        #because I made my markers slightly transparent, I need separate plots for lines and red markers
        #this avoids having the line become transparent
        #slightly transparent markers make it easier to see subtle differences between the lines
        #if you opt to set alpha at the default of 1 (not transparent), you can combine the first two red plots:
        #ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_r1, markerfacecolor=l_red,markersize=8, 
                    #linewidth=2,dashes=[2, 2,5,2], c='black')                
        #there's no built-in way to customise marker colours by variables, so the green markers always need a dummy line          
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_r1, markerfacecolor='none',markersize=8, 
                  linewidth=2,dashes=[2, 2,5,2], c='black')
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_r1, markerfacecolor=l_red, alpha=0.75, 
                markersize=8, c='none')
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gD', markevery=new_g1, markerfacecolor=l_green, alpha=0.75,
                markersize=8, c='none') 
        #square yellow markers show the predictions for time points which were withheld
        ax.plot(r.loc[0]+days[0], r.iloc[rows[i]],'-gs', markevery=markers_y, 
                markerfacecolor='yellow', alpha=0.9, markersize=11, c='none')
        #again, if you prefer alpha=1 you can combine the two lines for red markers:
        #ax.plot(days,rel.iloc[rows[i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red, markersize=8, 
                #linewidth=2, c='black')
        ax.plot(days, rel.iloc[rows[i]-1],'-gD', markevery=markers_r,markerfacecolor='none',markersize=8, 
                linewidth=2, c='black')
        ax.plot(days, rel.iloc[rows[i]-1],'-gD', markevery=markers_r,markerfacecolor=d_red, alpha=0.75, markersize=8, 
                c='none')
        ax.plot(days,rel.iloc[rows[i]-1],'-gD', markevery=markers_g,markerfacecolor=d_green,alpha=0.75, markersize=8, 
                c='none')
        #optional: insert code from Section 6 to add a legend for each plot - remember to make size/fit adjustments
        plt.title('{} Composition with Predictions'.format(key['Name'][rows[i]-1]), size=20)
        plt.xlabel("Age (Days) of Participant {}".format(ID), size=16)
        #alternative x axis label, if the participant's ID is in the title
        #plt.xlabel("Age(Days)", size=16)
        plt.ylabel("Relative Abundance", size=16)
        plt.setp(ax.get_xticklabels(), size=12)
        plt.setp(ax.get_yticklabels(), size=12)
        #the tight_layout function reduces white space in the image. 
        #If you turn off tight_layout you may need to adjust your text size etc.         
        plt.tight_layout()
        plt.savefig("Plots/{}b_pred_two.png".format(ID), format='png')
    plt.show()

In [ ]:
#test-run the function in a loop this time 
for name in IDs:
    plot_pred_two(name)

## Section 6: Legends
Here we have code for creating legends for the plots in this program to be saved as separate files. Then we provide a template code which can be copied and pasted into the functions, then adjusted accordingly to give every plot its own legend. 

In [ ]:
#create a legend for leave-k-out plots with predictions and save to a separate file using dummy plots
fig = plt.figure()
fig.patch.set_alpha(0.0)
ax = fig.add_subplot()
ax.plot([], [], linewidth=2, c='black',dashes=[2, 2,5,2], label="Noise-Free with Predictions")
ax.plot([], [], 'gD', color=l_red,alpha=0.75,label="Noise-Free Exacerbated")
ax.plot([], [], 'gD', color=l_green,alpha=0.75,label="Noise-Free Stable")
ax.plot([], [], 'gs', color='yellow',alpha=0.9, label="Prediction for Withheld Sample")
ax.plot([], [], linewidth=2, c='black', label="Observed")
ax.plot([], [], 'gD', color=d_red,alpha=0.75,label="Observed Exacerbated")
ax.plot([], [], 'gD', color=d_green,alpha=0.75,label="Observed Stable")
ax.legend(loc='center', shadow=True, ncol=2)
plt.gca().set_axis_off()
plt.savefig("Plots/Legend_with_Pred_b.png", format='png')
plt.show()

In [ ]:
#create a legend for plots without predictions and save to a separate file using dummy plots
#you may already have this legend saved from the program Plotsamples, in which case you can skip this
fig = plt.figure()
fig.patch.set_alpha(0.0)
ax = fig.add_subplot()
ax.plot([], [], linewidth=2, c='black',dashes=[2, 2,5,2], label="Noise-Free")
ax.plot([], [], 'gD', color=l_red,alpha=0.75,label="Noise-Free Exacerbated")
ax.plot([], [], 'gD', color=l_green,alpha=0.75,label="Noise-Free Stable")
ax.plot([], [], linewidth=2, c='black', label="Observed")
ax.plot([], [], 'gD', color=d_red,alpha=0.75,label="Observed Exacerbated")
ax.plot([], [], 'gD', color=d_green,alpha=0.75,label="Observed Stable")
ax.legend(loc='center', shadow=True, ncol=2)
plt.gca().set_axis_off()
plt.savefig("Plots/Basic_Legend.png", format='png')
plt.show()

In [ ]:
#code to paste into the functions at the indicated places - legends for leave-k-out plots with predictions
#it is written to place the legend outside the plot, where it won't interfere
#you may wish to change the position of the legend box or make other adjustments to the figsize, or make other edits
#it is created using dummy plots with the same features as our actual plots
ax.plot([], [], linewidth=2, c='black',dashes=[2, 2,5,2], label="Noise-Free with Predictions")
ax.plot([], [], 'gD', color=l_red,alpha=0.75,label="Noise-Free Exacerbated")
ax.plot([], [], 'gD', color=l_green,alpha=0.75,label="Noise-Free Stable")
ax.plot([], [], 'gs', color='yellow',alpha=0.9, label="Prediction for Withheld Sample")
ax.plot([], [], linewidth=2, c='black', label="Observed")
ax.plot([], [], 'gD', color=d_red,alpha=0.75,label="Observed Exacerbated")
ax.plot([], [], 'gD', color=d_green,alpha=0.75,label="Observed Stable")
chartBox = ax.get_position()
ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.6, chartBox.height])
#the tuple (1.25, 0.8) refers to the position relative to the width and height of the plot
ax.legend(loc='upper center', bbox_to_anchor=(1.25, 0.8), shadow=True, ncol=2)


In [ ]:
#code to paste into the functions at the indicated places - legends for plots without predictions
#it is written to place the legend outside the plot, where it won't interfere
#you may wish to change the position of the legend box or make other adjustments to the figsize, or make other edits
#it is created using dummy plots with the same features as our actual plots
ax.plot([], [], linewidth=2, c='black',dashes=[2, 2,5,2], label="Noise-Free")
ax.plot([], [], 'gD', color=l_red,alpha=0.75,label="Noise-Free Exacerbated")
ax.plot([], [], 'gD', color=l_green,alpha=0.75,label="Noise-Free Stable")
ax.plot([], [], linewidth=2, c='black', label="Observed")
ax.plot([], [], 'gD', color=d_red,alpha=0.75,label="Observed Exacerbated")
ax.plot([], [], 'gD', color=d_green,alpha=0.75,label="Observed Stable")
chartBox = ax.get_position()
ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.6, chartBox.height])
#the tuple (1.25, 0.8) refers to the position relative to the width and height of the plot
ax.legend(loc='upper center', bbox_to_anchor=(1.25, 0.8), shadow=True, ncol=2)